In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [2]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [3]:
class Network(nn.Module): 
    def __init__(self):                                                                
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self,t):
        t = F.relu(self.conv1(t))                          #简洁书写，将卷积与relu结合在一起
        t = F.max_pool2d(t,kernel_size=2,stride=2)     
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t,kernel_size=2,stride=2)
        
        t = F.relu(self.fc1(t.reshape(-1,12*4*4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t 

In [4]:
train_set = torchvision.datasets.FashionMNIST(
    root = 'data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([              #torchvision.transforms是pytorch中的图像预处理包。一般用Compose把多个步骤整合到一起
        transforms.ToTensor()
    ])
)

In [5]:
network = Network()                                                    

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)   
optimizer = optim.Adam(network.parameters(), lr=0.01)                  

total_loss = 0
total_correct = 0 
for batch in train_loader:                                                      #batch = next(iter(train_loader))                                       
    images, labels = batch                                                 

    preds = network(images)                                                
    loss = F.cross_entropy(preds, labels)   
    
    optimizer.zero_grad()                  #我们要告诉优化器把梯度属性中的权重清零，因为pytorch会积累梯度，防止累加
    loss.backward()                                                        
    optimizer.step()  
    
    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)
    
    
print('epoch:',0,'total_correct:',total_correct,'loss:',total_loss)




epoch: 0 total_correct: 47851 loss: 324.11772863566875


In [6]:
network = Network()                                                    

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)   
optimizer = optim.Adam(network.parameters(), lr=0.01)                  

for epoch in range(5):
    
    total_loss = 0
    total_correct = 0 
    
    for batch in train_loader:                                                      #batch = next(iter(train_loader))                                       
        images, labels = batch                                                 

        preds = network(images)                                                
        loss = F.cross_entropy(preds, labels)   

        optimizer.zero_grad()                  #我们要告诉优化器把梯度属性中的权重清零，因为pytorch会积累梯度，防止累加
        loss.backward()                                                        
        optimizer.step()  

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)


    print('epoch:',epoch,'total_correct:',total_correct,'loss:',total_loss)




epoch: 0 total_correct: 47243 loss: 333.41273602843285
epoch: 1 total_correct: 51006 loss: 238.43375098705292
epoch: 2 total_correct: 51647 loss: 223.45918083190918
epoch: 3 total_correct: 52039 loss: 212.95683501660824
epoch: 4 total_correct: 52225 loss: 207.3524653017521


In [7]:
total_correct/len(train_set)

0.8704166666666666